# Testing

## Imports

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Functions

In [2]:
def generate_random_rank_r_square_matrix_vector_b(m):
    r = int(np.floor(0.5 * m))
    while True:
        A = np.random.rand(m, m)
        U, S, VT = np.linalg.svd(A)
        S_bar = np.zeros((m, m))
        if S.shape[0] < r:
            continue
        for i in range(r):
            S_bar[i, i] = S[i]
        A = np.dot(U, np.dot(S_bar, VT))
        coefficients = np.random.rand(A.shape[1])
        b = np.dot(A, coefficients)
        return A, b

In [ ]:
def P1_matrix_frobenius_norm(A, H):
    AHA = np.dot(A, np.dot(H, A))
    return np.linalg.norm(AHA - A, ord="fro")

In [17]:
def problem_1_norm_PN_viable_solution(A, H, b, m):
    epsilon = 10 ** -5
    AHb = np.dot(A, np.dot(H, b))
    for i in range(m):
        if np.abs(AHb[i] - b[i]) > epsilon:
            print("AHb - b == 0 violation")
            return False
    return True

In [4]:
def problem_1_norm_P1_P4_viable_solution(A, H, m):
    epsilon = 10 ** -5
    AHA = np.dot(A, np.dot(H, A))
    for i in range(m):
        for j in range(m):
            if np.abs(AHA[i, j] - A[i, j]) > epsilon:
                return False
    HA_T = np.dot(H, A).T
    HA = np.dot(H, A)
    for i in range(m):
        for j in range(m):
            if np.abs(HA_T[i, j] - HA[i, j]) > epsilon:
                return False
    return True

In [5]:
def problem_1_norm_MSN_viable_solution(A, H, m):
    epsilon = 10 ** -5
    HAA_T = np.dot(H, np.dot(A, A.T))
    A_T = A.T
    for i in range(m):
        for j in range(m):
            if np.abs(HAA_T[i, j] - A_T[i, j]) > epsilon:
                return False
    return True

In [6]:
def problem_2_norm_KKT_solver(A, b):
    # Extracts matrix dimensions
    m = A.shape[0]

    # Creates a new model
    model = gp.Model("nlp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    H = []
    for i in range(m):
        H_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
        H.append(H_row)
    H = np.array(H)

    # Defines objective function
    objective = np.dot(np.dot(H, b).T, np.dot(H, b))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: AHb = b
    AHb = np.dot(A, np.dot(H, b))
    for i in range(m):
        model.addConstr(AHb[i] == b[i], name=f"constraint_eq_i={i}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        lambda_values = []
        for i in range(m):
            lambda_value = model.getConstrByName(f"constraint_eq_i={i}").Pi
            lambda_values.append(lambda_value)
        lambda_values = np.array(lambda_values)
        return lambda_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [13]:
def problem_1_norm_PN_solver(A, b):
    # Solves the following problem to get the lagrange multipliers for the PN equations
    lambda_values = problem_2_norm_KKT_solver(A=A, b=b)

    # Extracts matrix dimensions
    m = A.shape[0]

    # Defines auxiliary variables
    J = np.ones((m, m))

    # Creates a new model
    model = gp.Model("lp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    Z_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Z")
    H = []
    Z = []
    for i in range(m):
        H_row = []
        Z_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
            Z_row.append(Z_var[i, j])
        H.append(H_row)
        Z.append(Z_row)
    H = np.array(H)
    Z = np.array(Z)

    # Defines objective function
    objective = np.trace(np.dot(J.T, Z))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: Primal feasibility AHb = b
    AHb = np.dot(A, np.dot(H, b))
    for i in range(m):
        model.addConstr(AHb[i] == b[i], name=f"constraint_eq_1_i={i}")

    # Adds constraint: Z - H >= 0
    Z_minus_H = Z - H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_minus_H[i, j] >= 0, name=f"constraint_ineq_1_i={i}_j={j}")

    # Adds constraint: Z + H >= 0
    Z_plus_H = Z + H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_plus_H[i, j] >= 0, name=f"constraint_ineq_2_i={i}_j={j}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        H_values = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                H_values[i, j] = H[i, j].X
        return H_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [8]:
def problem_1_norm_P1_P4_solver(A, b):
    # Extracts matrix dimensions
    m = A.shape[0]

    # Defines auxiliary variables
    J = np.ones((m, m))

    # Creates a new model
    model = gp.Model("lp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    Z_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Z")
    H = []
    Z = []
    for i in range(m):
        H_row = []
        Z_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
            Z_row.append(Z_var[i, j])
        H.append(H_row)
        Z.append(Z_row)
    H = np.array(H)
    Z = np.array(Z)

    # Defines objective function
    objective = np.trace(np.dot(J.T, Z))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: P1 given by AHA = A
    AHA = np.dot(A, np.dot(H, A))
    for i in range(m):
        for j in range(m):
            model.addConstr(AHA[i, j] == A[i, j], name=f"constraint_P1_i={i}_j={j}")

    # Adds constraint: P4 given by (HA)^T = HA
    HA_T = np.dot(H, A).T
    HA = np.dot(H, A)
    for i in range(m):
        for j in range(m):
            model.addConstr(HA_T[i, j] == HA[i, j], name=f"constraint_P3_i={i}_j={j}")

    # Adds constraint: Z - H >= 0
    Z_minus_H = Z - H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_minus_H[i, j] >= 0, name=f"constraint_ineq_1_i={i}_j={j}")

    # Adds constraint: Z + H >= 0
    Z_plus_H = Z + H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_plus_H[i, j] >= 0, name=f"constraint_ineq_2_i={i}_j={j}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        H_values = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                H_values[i, j] = H[i, j].X
        return H_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [9]:
def problem_1_norm_MSN_solver(A, b):
    # Extracts matrix dimensions
    m = A.shape[0]

    # Defines auxiliary variables
    J = np.ones((m, m))

    # Creates a new model
    model = gp.Model("lp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    Z_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Z")
    H = []
    Z = []
    for i in range(m):
        H_row = []
        Z_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
            Z_row.append(Z_var[i, j])
        H.append(H_row)
        Z.append(Z_row)
    H = np.array(H)
    Z = np.array(Z)

    # Defines objective function
    objective = np.trace(np.dot(J.T, Z))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: MSN given by HAA^T = A^T
    HAA_T = np.dot(H, np.dot(A, A.T))
    A_T = A.T
    for i in range(m):
        for j in range(m):
            model.addConstr(HAA_T[i, j] == A_T[i, j], name=f"constraint_MSN_i={i}_j={j}")

    # Adds constraint: Z - H >= 0
    Z_minus_H = Z - H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_minus_H[i, j] >= 0, name=f"constraint_ineq_1_i={i}_j={j}")

    # Adds constraint: Z + H >= 0
    Z_plus_H = Z + H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_plus_H[i, j] >= 0, name=f"constraint_ineq_2_i={i}_j={j}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        H_values = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                H_values[i, j] = H[i, j].X
        return H_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [14]:
m = 80
A, b = generate_random_rank_r_square_matrix_vector_b(m=m)

In [15]:
lambda_values = problem_2_norm_KKT_solver(A=A, b=b)

In [12]:
print(lambda_values)

[ 0.45046644  0.50529744 -0.07739931 -0.24284038 -0.36457661  0.28964477
 -0.05092711 -0.54593193  0.74408876 -0.29668263  0.69063786 -0.12626541
 -0.1401758   0.48201469 -0.05676253 -0.7572246  -0.36626812 -0.16913132
 -0.         -0.43273369 -0.4730967  -0.20221864 -0.60413589 -0.
  0.45761725 -0.41598988  0.30958345  0.1610658   0.44327705 -0.01642673
  0.19336427 -0.85301736  0.2085342   0.384412    0.15238681  0.17191987
  1.31597991 -0.          0.28603204 -1.29439056 -0.08623482 -0.75860836
  0.97933123 -0.11700715  0.16309111  0.45144536 -0.01956408 -0.18222391
 -0.40543322  0.15481672 -0.          0.07132047  0.1192919  -0.03482935
 -0.51887689  0.55830946 -0.25374784 -0.09006133  0.37724901 -0.11252288
 -0.46764485 -0.         -0.05717788  0.06199351  0.57261185  0.29701639
 -0.50566119 -0.06859631  0.24126932  0.14173458  0.01370112 -0.21924968
 -0.62544181  0.75972024 -0.35505386  0.51205037  0.60130216  0.70902538
  0.52206442 -0.47992447]


In [19]:
H_star = problem_1_norm_PN_solver(A=A, b=b)
print(problem_1_norm_PN_viable_solution(A=A, H=H_star, b=b, m=m))
print(np.linalg.norm(H_star.flatten(), ord=1))
print(H_star)

True
1.6095088211394761
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [20]:
# Usando numpy.nonzero() para encontrar os índices dos elementos não nulos
indices = np.nonzero(H_star)

# Imprimindo os elementos não nulos
elementos_nao_nulos = H_star[indices]
print(elementos_nao_nulos)
print(elementos_nao_nulos.shape)
print(indices[1])

[0.04199111 0.07394229 0.0125317  0.02402991 0.14237563 0.02225225
 0.00331112 0.00383068 0.00799918 0.01708707 0.02314166 0.12577582
 0.00926735 0.06196385 0.0041961  0.07313717 0.08161118 0.00394497
 0.03608865 0.06681959 0.08968655 0.12819356 0.02844101 0.07762621
 0.00460713 0.02015199 0.03332219 0.00041175 0.00047572 0.00847392
 0.01459365 0.06175292 0.028501   0.02069163 0.03180918 0.0629191
 0.01991947 0.0140831  0.02590545 0.102646  ]
(40,)
[26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26
 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26]


In [ ]:
H_star = problem_1_norm_P1_P4_solver(A=A, b=b)
print(problem_1_norm_P1_P4_viable_solution(A=A, H=H_star, m=m))
print(np.linalg.norm(H_star.flatten(), ord=1))
print(H_star)

In [ ]:
H_star = problem_1_norm_MSN_solver(A=A, b=b)
print(problem_1_norm_MSN_viable_solution(A=A, H=H_star, m=m))
print(np.linalg.norm(H_star.flatten(), ord=1))
print(H_star)